In [2]:
from pyspark.sql.functions import explode

In [ ]:
from time import time

### Load raw JSON data from S3 into a Spark DataFrame

In [ ]:
s = time()
raw_tweets = spark.read.json("s3a://kinesis-lab5/gagliardi2017/02/*/*/*")
print time() - s

In [ ]:
# raw_tweets.printSchema()

In [ ]:
filtered_tweets = raw_tweets.filter('id IS NOT NULL')
filtered_tweets.cache()

### Write distinct Users table

In [ ]:
users = filtered_tweets.select('user.*').distinct()
users.write.parquet("s3a://gagliardilitme/users")

### Write restricted Tweets table

In [ ]:
tweet_df = filtered_tweets.selectExpr("id", 
                                      "BIGINT(timestamp_ms) AS timestamp_ms", 
                                      "user.id AS user_id", 
                                      "text")
tweet_df.write.parquet("s3a://gagliardilitme/tweets")

### Write normalized Hashtags table

In [ ]:
hashtags = filtered_tweets.select(raw_tweets.id, 
                                  explode(raw_tweets.entities.hashtags.text).alias('hashtag'))
hashtags.cache()
hashtags.write.parquet("s3a://gagliardilitme/hashtags")

### Derive hashtag counts

In [ ]:
hash_cnt = hashtags.groupBy("hashtag").count().orderBy("count", ascending=False)

In [ ]:
top20hash = hash_cnt.limit(20).toPandas()
top20hash

In [ ]:
%matplotlib inline

In [ ]:
top20hash.plot();